In [1]:
import os
import time
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from collections import OrderedDict
import PIL

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
dataset_location = '/home/marcin/Datasets/udacity-challange-flower-data/flower_data/'

# Load Model

In [4]:
model_pth = os.path.join(dataset_location, 'model.pth')
model_pth

'/home/marcin/Datasets/udacity-challange-flower-data/flower_data/model.pth'

In [5]:
model = models.densenet201(pretrained=True)

for param in model.parameters():
    param.requires_grad = False

/home/marcin/.anaconda/envs/ptgpu/lib/python3.7/site-packages/torchvision/models/densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight.data)


In [6]:
classifier = nn.Sequential(OrderedDict([
    ('bn1', nn.BatchNorm1d(1920)),
    ('fc1', nn.Linear(1920, 128)),
    ('elu1', nn.ELU()),
    ('drp1', nn.Dropout(0.75)),
    
#     ('fc2', nn.Linear(512, 512)),
#     ('elu2', nn.ELU()),
#     ('drp2', nn.Dropout(0.75)),
    
    ('fcf', nn.Linear(128, 102)),
]))
model.classifier = classifier

In [7]:
model.load_state_dict(torch.load(model_pth))

In [8]:
model.to(device);

# Load Model

In [ ]:
model_cnn = models.densenet201(pretrained=True)

for param in model_cnn.parameters():
    param.requires_grad = False

In [ ]:
model_pth = os.path.join(dataset_location, 'model.pth')
model_pth

In [ ]:
model = nn.Sequential(OrderedDict([
    ('bn1', nn.BatchNorm1d(1920)),
    ('fc1', nn.Linear(1920, 128)),
    ('elu1', nn.ELU()),
    ('drp1', nn.Dropout(0.75)),
    
#     ('fc2', nn.Linear(512, 512)),
#     ('elu2', nn.ELU()),
#     ('drp2', nn.Dropout(0.75)),
    
    ('fcf', nn.Linear(128, 102)),
]))

In [ ]:
model.load_state_dict(torch.load(model_pth))

In [ ]:
model_cnn.classifier = model

In [ ]:
model = model_cnn

In [ ]:
model.to(device)

# Test Model

**Create Dataloaders**

In [9]:
transforms_valid = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], 
                         [0.229, 0.224, 0.225])
])

In [10]:
dataset_valid = datasets.ImageFolder(os.path.join(dataset_location, 'valid'), transforms_valid)
dataset_test = datasets.ImageFolder(os.path.join(dataset_location, 'test'), transforms_valid)
print('Number of valid images:', len(dataset_valid))
print('Number of test images:', len(dataset_test))

Number of valid images: 818
Number of test images: 819


In [11]:
dataloader_valid = torch.utils.data.DataLoader(dataset_valid, batch_size=16, shuffle=True,
                                               num_workers=6, pin_memory=True)
dataloader_test = torch.utils.data.DataLoader(dataset_test, batch_size=16, shuffle=True,
                                               num_workers=6, pin_memory=True)

**Test Model**

In [12]:
def accuracy(logits, labels): 
    predictions = torch.argmax(logits, dim=1)
    return (predictions == labels).float().mean()  # tensor!!

In [13]:
hist = { 'valid_loss':[], 'valid_acc':[] }        # valid set loss/acc every epoch

In [16]:
criterion = nn.CrossEntropyLoss()

In [17]:
batch_size = 16

model.eval()

### Evaluate Valid Set ###
loss_sum = 0
acc_sum = 0
for images, labels in dataloader_valid:

    # Push to GPU
    x = images.to(device)
    y = labels.to(device)

    with torch.no_grad():
        outputs = model(x)
        loss = criterion(outputs, y)
        acc = accuracy(outputs, y)
        loss_sum += loss.item() * len(images)
        acc_sum += acc.item() * len(images)

valid_loss = loss_sum / len(dataset_valid)
valid_acc = acc_sum / len(dataset_valid)

In [18]:
print( '         loss / acc')
print(f'Valid: {valid_loss:6.4f} / {valid_acc:4.4f}')

         loss / acc
Valid: 0.1063 / 0.9804
